In [1]:
import chromadb

import json

In [2]:

# Step 1: Reinitialize the Chroma client
client = chromadb.PersistentClient(path="chroma_tmp")

# Step 2: Retrieve the existing collection
collection = client.get_collection(name="investopedia_articles")

# Step 3: Perform a search query
query_text = "What is a stock?"

# Perform the search query
results = collection.query(
    query_texts=[query_text],
    n_results=5  # Adjust the number of results you want to retrieve
)

# Print the search results
print("Search Results:")
for document, doc_id, metadata in zip(results["documents"], results["ids"], results["metadatas"]):
    print(f"ID: {doc_id}")
    print(f"Metadata: {metadata}")
    print(f"Document: {document}")
    print("-" * 80)
    print(results)

Add of existing embedding ID: e4e643210b05cc8b3deaf7af05817029
Add of existing embedding ID: 2c69ed878310348ef9ae960e8eb3a1c0


Search Results:
ID: ['1cd6502212e15745f4822cdb0639c254', 'b558218e5b87c0480fe5b22677c0060c', 'dc53b04c65550998824ae1433460b8eb', 'f0ae8c2bdfaab8f7da217d8d02f70867', 'd89fc993acd77342ae554add4a2d4b66']
Metadata: [{'element_id': '1cd6502212e15745f4822cdb0639c254', 'is_title': True, 'type': 'Title'}, {'element_id': 'b558218e5b87c0480fe5b22677c0060c', 'is_title': True, 'type': 'Title'}, {'element_id': 'dc53b04c65550998824ae1433460b8eb', 'is_title': True, 'type': 'Title'}, {'element_id': 'f0ae8c2bdfaab8f7da217d8d02f70867', 'is_title': True, 'type': 'Title'}, {'element_id': 'd89fc993acd77342ae554add4a2d4b66', 'is_title': True, 'type': 'Title'}]
Document: ['What Are Stocks?', 'What Are Stocks?', 'Stocks', 'Stocks', 'Stocks']
--------------------------------------------------------------------------------
{'ids': [['1cd6502212e15745f4822cdb0639c254', 'b558218e5b87c0480fe5b22677c0060c', 'dc53b04c65550998824ae1433460b8eb', 'f0ae8c2bdfaab8f7da217d8d02f70867', 'd89fc993acd77342ae554add4a2d4b66']],

In [3]:
# Function to retrieve narratives from the original JSON data
def get_narratives_for_id(element_id, all_cleaned_jsons):
    for record in all_cleaned_jsons:
        metadata = record.get('metadata')
        if metadata and metadata.get('element_id') == element_id:
            return record.get('data', [])
        for item in record.get('data', []):
            if item.get('element_id') == element_id:
                return item.get('narratives', [])
    return []

# Read the original JSON data
with open('all_cleaned_records.txt', 'r') as file:
    all_cleaned_jsons = json.load(file)

# Initialize the Chroma client
client = chromadb.PersistentClient(path="chroma_tmp")

# Load the existing collection
collection = client.get_collection(name="investopedia_articles")

# Perform a search query
query_text = "What is a stock?"
search_results = collection.query(
    query_texts=[query_text],
    n_results=5  # Adjust the number of results as needed
)

# Extract the results
documents = search_results["documents"]
ids = search_results["ids"]
metadatas = search_results["metadatas"]

# Print the results with narratives
print("Search Results:")
for doc, doc_id, meta in zip(documents, ids, metadatas):
    print(f"ID: {doc_id}")
    print(f"Metadata: {meta}")
    print(f"Document: {doc}")
    
    # Retrieve and print narratives
    narratives = get_narratives_for_id(doc_id[0], all_cleaned_jsons)
    if narratives:
        print("Narratives:")
        for narrative in narratives:
            print(f"- {narrative['text']}")
    print("-" * 80)


Search Results:
ID: ['1cd6502212e15745f4822cdb0639c254', 'b558218e5b87c0480fe5b22677c0060c', 'dc53b04c65550998824ae1433460b8eb', 'f0ae8c2bdfaab8f7da217d8d02f70867', 'd89fc993acd77342ae554add4a2d4b66']
Metadata: [{'element_id': '1cd6502212e15745f4822cdb0639c254', 'is_title': True, 'type': 'Title'}, {'element_id': 'b558218e5b87c0480fe5b22677c0060c', 'is_title': True, 'type': 'Title'}, {'element_id': 'dc53b04c65550998824ae1433460b8eb', 'is_title': True, 'type': 'Title'}, {'element_id': 'f0ae8c2bdfaab8f7da217d8d02f70867', 'is_title': True, 'type': 'Title'}, {'element_id': 'd89fc993acd77342ae554add4a2d4b66', 'is_title': True, 'type': 'Title'}]
Document: ['What Are Stocks?', 'What Are Stocks?', 'Stocks', 'Stocks', 'Stocks']
Narratives:
- A stock, also known as equity, is a security that represents the ownership of a fraction of the issuing corporation. Units of stock are called shares, which entitle the owner to a proportion of the corporation’s assets and profits equal to how much stock the

In [4]:
def load_json_in_chunks(file_path, chunk_size=100):
    """Load JSON data from a file in chunks."""
    with open(file_path, 'r') as file:
        data = json.load(file)
        for i in range(0, len(data), chunk_size):
            yield data[i:i + chunk_size]

# Initialize the Chroma client
client = chromadb.PersistentClient(path="chroma_tmp")

# Load the existing collection
collection = client.get_collection(name="investopedia_articles")

# Read the original JSON data incrementally
all_cleaned_jsons = []
for chunk in load_json_in_chunks('all_cleaned_records.txt'):
    all_cleaned_jsons.extend(chunk)

print(f"Total records loaded: {len(all_cleaned_jsons)}")


Total records loaded: 6288


In [5]:
def get_narratives_for_id(element_id, all_cleaned_jsons):
    # Search for the element ID in the JSON data
    for record in all_cleaned_jsons:
        # Check if the metadata element ID matches
        metadata = record.get('metadata')
        # If the metadata element ID matches, return the narratives
        if metadata and metadata.get('element_id') == element_id:
            # Return the narratives if found
            return record.get('data', [])
        # If the metadata element ID does not match, check the nested elements
        for item in record.get('data', []):
            # Check if the element ID matches
            if item.get('element_id') == element_id:
                # Return the narratives if found
                return item.get('narratives', [])
    # Return an empty list if no narratives are found
    return []

In [6]:
# Perform a search query on a subset
query_text = "where can I buy stocks"
# Perform the search query
search_results = collection.query(
    # Adjust the query text as needed
    query_texts=[query_text],
    # Adjust the number of results as needed
    n_results=5
)
all_cleaned_jsons = []

# Extract the results
documents = search_results["documents"]
ids = search_results["ids"]
metadatas = search_results["metadatas"]

# Print the results with narratives
print("Search Results:")
# Iterate over the search results
for doc, doc_id, meta in zip(documents, ids, metadatas):
    # Print the document, metadata, and ID
    print(f"ID: {doc_id}")
    print(f"Metadata: {meta}")
    print(f"Document: {doc}")
    
    # Retrieve and print narratives
    narratives = get_narratives_for_id(doc_id[0], all_cleaned_jsons)
    # Print the narratives if available
    if narratives:
        print("Narratives:")
        # Iterate over the narratives
        for narrative in narratives:
            print(f"- {narrative['text']}")
    print("-" * 80)


Search Results:
ID: ['9426d24f2101422f8dadbe1bc9a4a7c8', '0ef13e12a95eb4b3b5afa17a8b848cbf', 'f0ae8c2bdfaab8f7da217d8d02f70867', 'ca4d9de3b1ed2e9e0a6a3f0809e34646', 'd89fc993acd77342ae554add4a2d4b66']
Metadata: [{'element_id': '9426d24f2101422f8dadbe1bc9a4a7c8', 'is_title': True, 'type': 'Title'}, {'element_id': '0ef13e12a95eb4b3b5afa17a8b848cbf', 'is_title': True, 'type': 'Title'}, {'element_id': 'f0ae8c2bdfaab8f7da217d8d02f70867', 'is_title': True, 'type': 'Title'}, {'element_id': 'ca4d9de3b1ed2e9e0a6a3f0809e34646', 'is_title': True, 'type': 'Title'}, {'element_id': 'd89fc993acd77342ae554add4a2d4b66', 'is_title': True, 'type': 'Title'}]
Document: ['How Do You Buy Stock?', 'Stocks', 'Stocks', 'Stocks', 'Stocks']
--------------------------------------------------------------------------------


In [8]:
def perform_search(query_text, n_results=10, max_relevant_docs=3):
    # Perform the search query to get relevant documents
    search_results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )

    # Extract the results
    documents = search_results["documents"]
    ids = search_results["ids"]
    metadatas = search_results["metadatas"]

    # Print the results with narratives
    print("Search Results:")
    relevant_count = 0
    for doc, doc_id, meta in zip(documents, ids, metadatas):
        if relevant_count >= max_relevant_docs:
            break

        # Each result is returned as a list, so we need to access the first element
        doc = doc[0]
        doc_id = doc_id[0]
        meta = meta[0]

        print(f"ID: {doc_id}")
        print(f"Metadata: {meta}")
        print(f"Document: {doc}")

        # Check if the result is a title
        if meta['type'] == 'Title':
            # Print the title
            print(f"Title: {doc}")

            # Find related NarrativeText entries by querying the collection again
            narrative_search_results = collection.get(
                where={"parent_title_id": doc_id}
            )

            # Extract the narrative documents
            narrative_documents = narrative_search_results["documents"]

            # Print the related narratives
            if narrative_documents:
                print("Narratives:")
                for narrative in narrative_documents:
                    # Access the first element of the narrative list
                    narrative_text = ''.join(narrative)
                    print(f"- {narrative_text}")

        relevant_count += 1
        print("-" * 80)

In [9]:
query_text = "What is a Bitcoin?"
perform_search(query_text, n_results=10, max_relevant_docs=3)


Search Results:
ID: 99919fe9440b6c8804dc51eca326bdd8
Metadata: {'element_id': '99919fe9440b6c8804dc51eca326bdd8', 'is_title': True, 'type': 'Title'}
Document: What Is Bitcoin?
Title: What Is Bitcoin?
Narratives:
- Read on to learn more about the cryptocurrency that started it all—the history behind it, how to mine it, buy it, and what it can be used for.
- Bitcoin was introduced to the public in 2009 by an anonymous developer or group of developers using the name Satoshi Nakamoto. It has since become the most well-known and largest cryptocurrency in the world. Its popularity has inspired the development of many other cryptocurrencies.
- Bitcoin (BTC) is a cryptocurrency (a virtual currency) designed to act as money and a form of payment outside the control of any one person, group, or entity. This removes the need for trusted third-party involvement (e.g., a mint or bank) in financial transactions. It is rewarded to blockchain miners who verify transactions and can be purchased on seve

In [10]:
def perform_search(query_text, n_results=10, max_relevant_docs=3):
    database_paths = [f"chroma_tmp_part{i}" for i in range(1, 11)]  # Adjust the range as needed
    aggregated_results = []

    for db_path in database_paths:
        client = chromadb.PersistentClient(path=db_path, settings=chromadb.Settings(allow_reset=True))
        collection = client.get_collection(name=f"investopedia_articles_part{database_paths.index(db_path) + 1}")
        
        # Perform the search query to get relevant documents
        search_results = collection.query(
            query_texts=[query_text],
            n_results=n_results
        )

        # Extract the results
        documents = search_results["documents"]
        ids = search_results["ids"]
        metadatas = search_results["metadatas"]

        for doc, doc_id, meta in zip(documents, ids, metadatas):
            aggregated_results.append((doc[0], doc_id[0], meta[0], db_path))

    # Print the results with narratives
    print("Search Results:")
    relevant_count = 0
    for doc, doc_id, meta, db_path in aggregated_results:
        if relevant_count >= max_relevant_docs:
            break

        print(f"ID: {doc_id}")
        print(f"Metadata: {meta}")
        print(f"Document: {doc}")

        # Check if the result is a title
        if meta['type'] == 'Title':
            # Print the title
            print(f"Title: {doc}")

            # Find related NarrativeText entries by querying the collection again
            client = chromadb.PersistentClient(path=db_path, settings=chromadb.Settings(allow_reset=True))
            collection = client.get_collection(name=f"investopedia_articles_part{database_paths.index(db_path) + 1}")
            narrative_search_results = collection.get(
                where={"parent_title_id": doc_id}
            )

            # Extract the narrative documents
            narrative_documents = narrative_search_results["documents"]

            # Print the related narratives
            if narrative_documents:
                print("Narratives:")
                for narrative in narrative_documents:
                    # Access the first element of the narrative list
                    narrative_text = ''.join(narrative)
                    print(f"- {narrative_text}")

        relevant_count += 1
        print("-" * 80)

In [11]:
query_text = "What is a Bitcoin?"
perform_search(query_text, n_results=10, max_relevant_docs=3)

Add of existing embedding ID: 1f95f3058460d8684b842a4535f2b781
Add of existing embedding ID: 95d47d19d8732f9278546f4babc3d3c5
Add of existing embedding ID: 3bdcd2d9dd1fe70311b1d336669ae265
Add of existing embedding ID: 6a69c0b3d41f28fe401364bd41a5e1f9
Add of existing embedding ID: e4e643210b05cc8b3deaf7af05817029
Add of existing embedding ID: 4f4378092bcda9fce5bb25576d6971c7
Add of existing embedding ID: 7788ff24973ba848d15c9515c2155872
Add of existing embedding ID: 791ea7d626c3425158ebceae149ec950
Add of existing embedding ID: 71f3af46269044ea523007404155a42f
Add of existing embedding ID: b9ecb04c84766cde5b519ff4cd4365bc
Add of existing embedding ID: e4e643210b05cc8b3deaf7af05817029
Add of existing embedding ID: 6a69c0b3d41f28fe401364bd41a5e1f9
Add of existing embedding ID: 9d5e4e792ab5be25fab6475c892c3e8a
Add of existing embedding ID: 669a157b4928b276af0f25eaf07a6290
Add of existing embedding ID: e4e643210b05cc8b3deaf7af05817029
Add of existing embedding ID: e4e643210b05cc8b3deaf7af0

Search Results:
ID: 99919fe9440b6c8804dc51eca326bdd8
Metadata: {'element_id': '99919fe9440b6c8804dc51eca326bdd8', 'is_title': True, 'type': 'Title'}
Document: bitcoin
Title: bitcoin
Narratives:
- read learn cryptocurr start all—th histori behind mine buy use
- bitcoin introduc public 2009 anonym develop group develop use name satoshi nakamoto sinc becom wellknown largest cryptocurr world popular inspir develop mani cryptocurr
- bitcoin btc cryptocurr virtual currenc design act money form payment outsid control one person group entiti remov need trust thirdparti involv eg mint bank financi transact reward blockchain miner verifi transact purchas sever exchang
--------------------------------------------------------------------------------
ID: bb02679ea98403c97763e8f738c61332
Metadata: {'element_id': 'bb02679ea98403c97763e8f738c61332', 'is_title': True, 'type': 'Title'}
Document: bitcoin token coin
Title: bitcoin token coin
Narratives:
- bitcoin coin also known cryptocurr use trade store